# 人脸采集

In [1]:
import cv2
import os
import re
import time  # 用于控制采集间隔

# 创建存储人脸数据的根文件夹
if not os.path.exists('known_faces'):
    os.makedirs('known_faces')

# 提前初始化人脸检测器
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

# 函数：获取下一个person序号（自动递增）
def get_next_person_id():
    dirs = [d for d in os.listdir('known_faces') if os.path.isdir(os.path.join('known_faces', d))]
    max_id = 0
    pattern = re.compile(r'^person(\d+)$')  # 匹配person+数字格式
    for d in dirs:
        match = pattern.match(d)
        if match:
            current_id = int(match.group(1))
            if current_id > max_id:
                max_id = current_id
    return max_id + 1

# 获取当前采集的person名称
next_id = get_next_person_id()
current_person = f'person{next_id}'
save_dir = f'known_faces/{current_person}'
os.makedirs(save_dir, exist_ok=True)  # 创建文件夹（已存在则不报错）

# 打开摄像头
cap = cv2.VideoCapture(0)
count = 0
print(f'正在采集 {current_person} 的人脸样本（共20张，每张间隔1秒，按q结束）...')

while count < 20:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 人脸检测
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # 检测到人脸时才保存（确保每张都是有效人脸）
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # 取第一个检测到的人脸
        # 裁剪并统一尺寸
        face_img = gray[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (200, 200))
        # 保存图片
        cv2.imwrite(f'{save_dir}/{count}.jpg', face_img)
        count += 1
        # 实时显示进度
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'已采集 {count}/20（间隔1秒）', (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f'已采集 {count}/20 张')
        
        # 强制等待1秒（确保间隔）
        time.sleep(1)
    
    # 显示画面
    cv2.imshow('Collect Faces', frame)
    
    # 按q键提前退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
cv2.destroyAllWindows()
print(f'{current_person} 采集完成，共 {count} 张样本')

正在采集 person1 的人脸样本（共20张，每张间隔1秒，按q结束）...
已采集 1/20 张
已采集 2/20 张
已采集 3/20 张
已采集 4/20 张
已采集 5/20 张
已采集 6/20 张
已采集 7/20 张
已采集 8/20 张
已采集 9/20 张
已采集 10/20 张
已采集 11/20 张
已采集 12/20 张
已采集 13/20 张
已采集 14/20 张
已采集 15/20 张
已采集 16/20 张
已采集 17/20 张
已采集 18/20 张
已采集 19/20 张
已采集 20/20 张
person1 采集完成，共 20 张样本


# 训练

In [5]:
import cv2
import os
import numpy as np

# 加载训练数据
faces = []
labels = []
label_map = {}  # 姓名到标签的映射
person_names = os.listdir('known_faces') if os.path.exists('known_faces') else []

for label, name in enumerate(person_names):
    label_map[label] = name
    person_dir = f'known_faces/{name}'
    
    # 数据校验：检查样本文件夹是否存在
    if not os.path.exists(person_dir):
        print(f"警告：未找到 {name} 的样本文件夹 {person_dir}，已跳过")
        continue
    
    # 数据校验：检查文件夹是否为空
    img_files = os.listdir(person_dir)
    if not img_files:
        print(f"警告：{name} 的样本文件夹为空，已跳过")
        continue
    
    for img_name in img_files:
        img_path = f'{person_dir}/{img_name}'
        # 尝试读取图片，增加错误处理
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"警告：无法读取图片 {img_path}，已跳过")
                continue
            faces.append(img)
            labels.append(label)
        except Exception as e:
            print(f"处理 {img_path} 时出错：{str(e)}，已跳过")

# 检查是否有有效训练数据
if not faces or not labels:
    print("错误：没有有效的训练数据，请先正确采集人脸样本")
else:
    # 训练LBPH人脸识别模型
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(labels))
    recognizer.save('face_recognizer.yml')
    print(f'模型训练完成，已知人员：{label_map}')

警告：.ipynb_checkpoints 的样本文件夹为空，已跳过
模型训练完成，已知人员：{0: '.ipynb_checkpoints', 1: 'jbx', 2: 'liuxin', 3: 'person1'}


# 监控

In [6]:
import cv2
import os
import time
import winsound
import numpy as np
from datetime import datetime

# 配置参数
ALARM_THRESHOLD = 3  # 连续检测到陌生人脸的次数阈值
KNOWN_THRESHOLD = 100  # 人脸识别置信度阈值（越低越严格）
ALARM_SOUND = 'alarm.wav'  # 警报声音文件（需提前准备）

# 初始化组件
recognizer = cv2.face.LBPHFaceRecognizer_create()
try:
    recognizer.read('face_recognizer.yml')
except:
    print("警告：未找到训练模型，将无法进行人脸识别")

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None  # 视频写入器

# 状态变量
motion_detected = False
stranger_count = 0
static_back = None
recording = False
label_map = {}  # 从训练模型中加载的姓名映射
is_stranger = False  # 新增：标记当前帧是否检测到「不符合阈值的陌生人」

# 从训练数据中重建label_map
if os.path.exists('known_faces'):
    person_names = os.listdir('known_faces')
    for label, name in enumerate(person_names):
        label_map[label] = name

def trigger_alarm():
    """触发警报（仅保留声音警报）"""
    # 声音警报
    if os.path.exists(ALARM_SOUND):
        # 循环播放警报声，直到运动停止
        winsound.PlaySound(ALARM_SOUND, winsound.SND_LOOP + winsound.SND_ASYNC)
    else:
        # 若没有alarm.wav文件，使用系统蜂鸣声
        winsound.Beep(1000, 2000)  # 1000Hz频率，持续2秒

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 每帧初始化「陌生人标记」（避免上一帧状态残留）
    is_stranger = False
    
    # 运动检测
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if static_back is None:
        static_back = gray
        continue
    
    # 计算帧差异
    diff_frame = cv2.absdiff(static_back, gray)
    thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)
    
    # 检测运动轮廓（判断是否有「有效运动」：面积>5000）
    contours, _ = cv2.findContours(thresh_frame.copy(), 
                                  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    motion_detected = any(cv2.contourArea(c) > 5000 for c in contours)  # 过滤小轮廓
    
    # 人脸识别（仅在检测到有效运动时执行）
    if motion_detected:
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            try:
                label, confidence = recognizer.predict(face_roi)
            except:
                # 模型未加载时默认视为「不符合阈值的陌生人」
                label, confidence = -1, 1000
            
            # 判断是否为「不符合阈值的陌生人」（置信度>KNOWN_THRESHOLD）
            if confidence > KNOWN_THRESHOLD:
                name = "Stranger"
                is_stranger = True  # 标记为「不符合阈值」
                stranger_count += 1
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)  # 红色框标记陌生人
                # 触发警报（连续3次检测到陌生人）
                if stranger_count >= ALARM_THRESHOLD:
                    trigger_alarm()
            else:
                name = label_map.get(label, "Unknown")
                is_stranger = False  # 标记为「符合阈值（熟人）」
                stranger_count = 0  # 重置陌生人计数
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)  # 绿色框标记已知人员
            
            cv2.putText(frame, f"{name} ({confidence:.1f})", (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # ========== 核心修改：录制触发/停止逻辑 ==========
    # 启动录制：有效运动（面积>5000） + 不符合阈值（陌生人） + 未录制
    if motion_detected and is_stranger and not recording:
        recording = True
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        out = cv2.VideoWriter(f'security_{timestamp}.mp4', fourcc, 20.0, (640, 480))
        print(f"开始录制：security_{timestamp}.mp4")
    
    # 停止录制：
    # 条件1：符合阈值（熟人） + 正在录制；
    # 条件2：无有效运动 + 正在录制（兜底逻辑）
    if ((not is_stranger and motion_detected) or not motion_detected) and recording:
        recording = False
        if out:
            out.release()
            out = None
            stranger_count = 0  # 重置陌生人计数
            winsound.PlaySound(None, winsound.SND_PURGE)  # 停止警报
        print("停止录制")
    
    # 写入视频帧（仅在录制状态且写入器有效时）
    if recording and out is not None:
        out.write(frame)
    
    # 显示监控画面
    cv2.imshow('Security Monitor', frame)
    
    # 按q退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 资源释放
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
winsound.PlaySound(None, winsound.SND_PURGE)  # 确保警报停止

开始录制：security_20251221_162458.mp4
停止录制
开始录制：security_20251221_162458.mp4
停止录制
开始录制：security_20251221_162500.mp4
停止录制
开始录制：security_20251221_162503.mp4
停止录制
开始录制：security_20251221_162505.mp4
停止录制
开始录制：security_20251221_162505.mp4
停止录制
开始录制：security_20251221_162506.mp4
停止录制
